In [ ]:
import cv2
import torch
import easyocr

# Load the trained YOLO model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt', source='local')

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

In [ ]:
def detect_license_plate(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Perform YOLO detection
    results = model(img)
    plates = results.pandas().xyxy[0]  # Extract detection data as a pandas dataframe

    for _, row in plates.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        plate_img = img[y1:y2, x1:x2]

        # Use EasyOCR to recognize text from the license plate image
        text_result = reader.readtext(plate_img)
        for text in text_result:
            license_plate = text[1]
            print(f'Detected License Plate: {license_plate}')

            # Draw bounding box and label
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, license_plate, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the image with detected plates
    cv2.imshow('Detected Plates', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# Example usage
detect_license_plate('/dataset/test/images/6-cdmx2018aa_jpg.rf.9aedbc7ff702d75592ff33a4ebe4e99e.jpg')